In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
!pip install transformers accelerate bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.2 MB/s eta 0:00:00


In [30]:
!pip install -U transformers accelerate bitsandbytes -q

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import os, csv

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 136.8 MB/s eta 0:00:00


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [42]:
def generate_utf8_from_binary(binary_path: str, utf8_path: str):
    with open(binary_path, "rb") as f:
        binary = f.read()

    try:
        utf8_decoded = binary.decode("utf-8", errors="ignore")
    except UnicodeDecodeError:
        utf8_decoded = binary.decode("euc-kr", errors="ignore")

    with open(utf8_path, "w", encoding="utf-8") as f:
        f.write(utf8_decoded)

    print(f"텍스트 디코딩 파일 저장 완료: {utf8_path}")


In [70]:
import csv
import os
import string

# 설정값
BINARY_FILE_PATH = "/content/drive/MyDrive/Mentoring/data.img"
DECODED_TEXT_PATH = "/content/drive/MyDrive/Mentoring/decoded.txt"
CSV_OUTPUT_PATH = "/content/drive/MyDrive/Mentoring/csvout.csv"
CHUNK_SIZE = 3000

# 필터링 기준
def is_informative(text: str) -> bool:
    if len(text.strip()) < 20:
        return False
    visible_ratio = sum(c in string.printable for c in text) / len(text)
    if visible_ratio < 0.5:
        return False
    alnum_ratio = sum(c.isalnum() for c in text) / len(text)
    if alnum_ratio < 0.2:
        return False
    return True

# 기존 CSV로부터 완료된 청크 번호 확인
def get_completed_chunks(csv_path: str) -> set:
    if not os.path.exists(csv_path):
        return set()
    with open(csv_path, "r", encoding="utf-8") as f:
        reader = csv.reader(f)
        next(reader, None)  # 헤더 건너뜀
        return {int(row[0]) for row in reader if row and row[0].isdigit()}

# 분석 함수
def analyze_text_by_chunk(text_path: str, chunk_size: int, csv_path: str):
    with open(text_path, "r", encoding="utf-8") as f:
        text = f.read()

    total_chunks = (len(text) + chunk_size - 1) // chunk_size
    completed = get_completed_chunks(csv_path)

    # CSV 파일이 없다면 헤더 생성
    write_header = not os.path.exists(csv_path)
    with open(csv_path, "a", newline='', encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        if write_header:
            writer.writerow(["Chunk Index", "의미 있는 문자열 및 설명"])

        for i in range(total_chunks):
            if i in completed:
                continue

            chunk = text[i * chunk_size : (i + 1) * chunk_size]
            if not is_informative(chunk):
                print(f"{i+1}/{total_chunks}번 chunk는 무의미해 스킵함.")
                continue

            prompt = f"""### 지시문:
아래 텍스트는 펌웨어 바이너리에서 추출된 문자열입니다.
이 중에서 포렌식 분석에서 의미를 가질 수 있는 문자열을 찾아주세요.
- 사용자 정보, 설정 이름, 장치 상태값
- 보안 관련 설정, 디버깅 정보
- 시스템 명령어, URL, 경로, 버전, 서비스 이름
- 통신 프로토콜 관련 키워드 (CAN, HTTP, BLE 등)
- 개인정보 또는 설정 값처럼 보이는 문자열
- 평범하지 않고 의도된 정보처럼 보이는 문자열

**[요청 형식]**
아래와 같이 결과를 목록으로 제시해주세요:

1. [문자열] - 의미 및 설명
2. [문자열] - 의미 및 설명
...

### 텍스트:
{chunk}

### 결과:"""

            print(f"\n{i+1}/{total_chunks}번 chunk 분석 중...\n")
            try:
                output = pipe(prompt, max_new_tokens=600, do_sample=False)
                answer = output[0]["generated_text"].split("### 결과:")[-1].strip()
                print(f"분석 결과:\n{answer}\n")
                writer.writerow([i, answer])
            except Exception as e:
                print(f"❌ {i}번 chunk 처리 중 오류 발생: {e}")


In [ ]:
analyze_text_by_chunk(DECODED_TEXT_PATH, CHUNK_SIZE, CSV_OUTPUT_PATH)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1/248990번 chunk는 무의미해 스킵함.
2/248990번 chunk는 무의미해 스킵함.
3/248990번 chunk는 무의미해 스킵함.
4/248990번 chunk는 무의미해 스킵함.
5/248990번 chunk는 무의미해 스킵함.
6/248990번 chunk는 무의미해 스킵함.
7/248990번 chunk는 무의미해 스킵함.
8/248990번 chunk는 무의미해 스킵함.
9/248990번 chunk는 무의미해 스킵함.
10/248990번 chunk는 무의미해 스킵함.
11/248990번 chunk는 무의미해 스킵함.
12/248990번 chunk는 무의미해 스킵함.
13/248990번 chunk는 무의미해 스킵함.
14/248990번 chunk는 무의미해 스킵함.
15/248990번 chunk는 무의미해 스킵함.
16/248990번 chunk는 무의미해 스킵함.
17/248990번 chunk는 무의미해 스킵함.
18/248990번 chunk는 무의미해 스킵함.
19/248990번 chunk는 무의미해 스킵함.
20/248990번 chunk는 무의미해 스킵함.
21/248990번 chunk는 무의미해 스킵함.
22/248990번 chunk는 무의미해 스킵함.
23/248990번 chunk는 무의미해 스킵함.
24/248990번 chunk는 무의미해 스킵함.
25/248990번 chunk는 무의미해 스킵함.
26/248990번 chunk는 무의미해 스킵함.
27/248990번 chunk는 무의미해 스킵함.
28/248990번 chunk는 무의미해 스킵함.
29/248990번 chunk는 무의미해 스킵함.
30/248990번 chunk는 무의미해 스킵함.
31/248990번 chunk는 무의미해 스킵함.
32/248990번 chunk는 무의미해 스킵함.
33/248990번 chunk는 무의미해 스킵함.
34/248990번 chunk는 무의미해 스킵함.
35/248990번 chunk는 무의미해 스킵함.
36/248990번 chunk는 무의미해 스킵함.
3

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


분석 결과:
1. `android` - Android OS의 이름
2. `jackpal.androidterm.permission.APPEND_TO_PATH` - AndroidTerm 앱의 설정 이름
3. `com.android.providers.downloads` - Android Downloads Provider의 패키지 이름
4. `android.permission.SEND_DOWNLOAD_COMPLETED_INTENTS` - 다운로드 완료 이벤트 전송 권한
5. `android.permission.TMSC_SEND` - SMS 전송 권한
6. `android.permission.BIND_DIRECTORY_SEARCH` - 디렉터리 검색 서비스 바인딩 권한
7. `android.permission.SERIAL_PORT` - 시리얼 포트 액세스 권한
8. `android.permission.SEND_SMS` - SMS 전송 권한
9. `android.permission.CONFIGURE_WIFI_DISPLAY` - WiFi 디스플레이 설정 권한
10. `android.permission.HARDWARE_TEST` - 하드웨어 테스트 권한
11. `android.permission.CAPTURE_AUDIO_OUTPUT` - 오디오 출력 캡처 권한
12. `android.permission.ACCESS_CHECKIN_PROPERTIES` - 프로퍼티 확인 액세스 권한
13. `android.permission.KILL_BACKGROUND_PROCESSES` - 백그라운드 프로세스 종료 권한
14. `android.permission.CRYPT_KEEPER` - 암호화 저장소 액세스 권한
15. `broadcom.permission.BLUETOOTH_MAP` - 블루투스 맵핑 권한
16. `com.android.bluetooth` - Android Bluetooth 서비스 패키지 이름
17. `android.permission.ACCESS_LOCATION_EXTR

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

분석 결과:
1. `TableMemoryPoint_Table` - Memory Point Table, containing information about memory points
2. `NonSearch_Table` - Non-search table, containing information about non-searchable data
3. `NaviVersion` - Navigation version table, containing information about navigation software version
4. `State_Table` - State table, containing information about the current state of the device
5. `Search_Table` - Search table, containing information about searchable data
6. `NewMyMenu_Table` - New my menu table, containing information about custom menu items
7. `NaviMenu_ButtonState_Table` - Navigation menu button state table, containing information about navigation menu button states
8. `RegistPnt_SectorName_Table` - Registered point sector name table, containing information about registered points and their sector names
9. `RegistPnt_Special_Table` - Registered point special table, containing information about registered points with special attributes
10. `RegistPnt_G1_Table`, `RegistPnt_G2_Tabl

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


분석 결과:
1. `W` - 의미 및 설명: 펌웨어 바이너리에서 추출된 텍스트의 시작 부분. 의미는 불명.
2. `MM` - 의미 및 설명: 펌웨어 바이너리에서 추출된 텍스트의 시작 부분. 의미는 불명.
3. ` Seoul` - 의미 및 설명: 장치의 위치 정보. 서울시가 포함된 텍스트.
4. `SQLite format 3` - 의미 및 설명: 데이터베이스 형식. SQLite 3.0 이상을 사용하는 데이터베이스.
5. `CREATE TABLE` - 의미 및 설명: 데이터베이스 테이블 생성 명령어. 다양한 테이블이 생성되는 것을 확인할 수 있다.
6. `MemoryPoint_Table` - 의미 및 설명: 장치의 메모리 포인트 정보를 저장하는 테이블.
7. `NonSearch_Table` - 의미 및 설명: 장치의 비검색 정보를 저장하는 테이블.
8. `NaviVersion` - 의미 및 설명: 장치의 네비게이션 버전 정보를 저장하는 테이블.
9. `State_Table` - 의미 및 설명: 장치의 상태 정보를 저장하는 테이블.
10. `Search_Table` - 의미 및 설명: 장치의 검색 정보를 저장하는 테이블.
11. `NewMyMenu_Table` - 의미 및 설명: 장치의 마이 메뉴 정보를 저장하는 테이블.
12. `NaviMenu_ButtonState_Table` - 의미 및 설명: 장치의 네비게이션 메뉴 버튼 상태 정보를 저장하는 테이블.
13. `RegistPnt_SectorName_Table` - 의미 및 설명: 장치의 등록 포인트 섹터 이름 정보를 저장하는 테이블.
14. `RegistPnt_Special_Table` - 의미 및 설명: 장치의 등록 포인트 특별 정보를 저장하는 테이블.

위의 결과에서, 다음과 같은 정보들이 추출되었다.

* 장치의 위치 정보 (Seoul)
* 데이터베이스 형식 (SQLite 3)
* 다양한 테이블 생성 (MemoryPoint_Table, NonSearch_Table, NaviVersion, State_Ta

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


분석 결과:
1. `Info_Name STRING` - 사용자 정보 (이름)
2. `Info_Address STRING` - 사용자 정보 (주소)
3. `Info_Tel STRING` - 사용자 정보 (전화번호)
4. `Info_Longitude INT` - 지리적 정보 (경도)
5. `Info_Latitude INT` - 지리적 정보 (위도)
6. `Info_Distance STRING` - 장치 상태값 (거리)
7. `Info_Direction INT` - 장치 상태값 (방향)
8. `Info_Unit STRING` - 설정 이름 (단위)
9. `Info_SelectRegion STRING` - 설정 이름 (선택 지역)
10. `Info_RtResumeStop INT` - 설정 이름 (Resume Route Stop)
11. `Info_Longitude DOUBLE` - 지리적 정보 (경도, 더블)
12. `Info_Latitude DOUBLE` - 지리적 정보 (위도, 더블)
13. `Info_Oilprice STRING` - 설정 이름 (주유 가격)
14. `Info_Prev_Map INT` - 설정 이름 (이전 지도)
15. `Info_Route INT` - 설정 이름 (경로)
16. `Info_Loadtour INT` - 설정 이름 (투어 로드)
17. `Info_TpegBookMark INT` - 설정 이름 (TPEG 북마크)
18. `Info_TpegNotice INT` - 설정 이름 (TPEG 공지)
19. `Info_TpegAccident INT` - 설정 이름 (TPEG 사고)
20. `Info_TpegNews INT` - 설정 이름 (TPEG 뉴스)
21. `Info_RtResumeStop_toggle INT` - 설정 이름 (Resume Route Stop toggle)
22. `Info_ResumeRoute INT` - 설정 이름 (Resume Route)
23. `Info_SectorName1 STRING` - 설정 이름 (섹터 이름

In [67]:
def query_llm_from_csv(csv_path: str, user_query: str):
    import pandas as pd

    # CSV 로드
    df = pd.read_csv(csv_path)

    # 모든 분석 결과 텍스트를 합침
    combined_text = "\n\n".join(df["의미 있는 문자열 및 설명"].astype(str))

    # 프롬프트 구성
    prompt = f"""### 지시문:
아래는 펌웨어 분석 결과로 추출된 의미 있는 문자열과 설명입니다.
사용자의 요청은 다음과 같습니다: "{user_query}"

아래 내용을 바탕으로 요청에 맞는 정보를 최대한 정확하게 추출해 주세요.

### 내용:
{combined_text}

### 결과:"""

    # LLM 호출
    output = pipe(prompt, max_new_tokens=600, do_sample=False)
    answer = output[0]["generated_text"].split("### 결과:")[-1].strip()

    # 결과 출력
    print("📌 사용자 요청에 대한 응답:\n", answer)
    return answer


In [68]:
query_llm_from_csv(CSV_OUTPUT_PATH, "연락처만 모두 정리해서 출력해줘")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


📌 사용자 요청에 대한 응답:
 The results of the analysis are as follows:

1. `android.permission.GET_APP_OPS_STATS` - Android permission for getting app operations statistics
2. `com.hkmc.permission.ACCESS_RADIO` - Permission for accessing radio functionality
3. `android.permission.BIND_DEVICE_ADMIN` - Android permission for binding device admin
4. `android.permission.READ_CALL_LOG` - Android permission for reading call log
5. `android.permission.SUBSCRIBED_FEEDS_WRITE` - Android permission for writing subscribed feeds
6. `com.hkmc.permission.ACCESS_IPOD` - Permission for accessing iPod functionality
7. `com.hkmc.permission.ACCESS_MOF20` - Permission for accessing MOF20 functionality
8. `android.permission.CLEAR_APP_USER_DATA` - Android permission for clearing app user data
9. `android.permission.READ_SOCIAL_STREAM` - Android permission for reading social stream
10. `android.permission.MEDIA_CONTENT_CONTROL` - Android permission for controlling media content
11. `android.permission.BIND_INPUT_MET

"The results of the analysis are as follows:\n\n1. `android.permission.GET_APP_OPS_STATS` - Android permission for getting app operations statistics\n2. `com.hkmc.permission.ACCESS_RADIO` - Permission for accessing radio functionality\n3. `android.permission.BIND_DEVICE_ADMIN` - Android permission for binding device admin\n4. `android.permission.READ_CALL_LOG` - Android permission for reading call log\n5. `android.permission.SUBSCRIBED_FEEDS_WRITE` - Android permission for writing subscribed feeds\n6. `com.hkmc.permission.ACCESS_IPOD` - Permission for accessing iPod functionality\n7. `com.hkmc.permission.ACCESS_MOF20` - Permission for accessing MOF20 functionality\n8. `android.permission.CLEAR_APP_USER_DATA` - Android permission for clearing app user data\n9. `android.permission.READ_SOCIAL_STREAM` - Android permission for reading social stream\n10. `android.permission.MEDIA_CONTENT_CONTROL` - Android permission for controlling media content\n11. `android.permission.BIND_INPUT_METHOD` 

# 아래는 다른 모델

In [ ]:
!pip install -U transformers accelerate bitsandbytes -q

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
import binascii
import csv

BINARY_FILE_PATH ="/content/drive/MyDrive/Mentoring/data.img"          # 업로드한 바이너리
ASCII_DUMP_PATH = "/content/ascii_dump.txt"         # 변환된 ASCII 파일
CSV_OUTPUT_PATH = "/content/final_analysis.csv"     # 분석 결과 저장
QUERY = "이름, 이메일 주소, 전화번호를 모두 찾아줘"

In [ ]:
def generate_ascii_from_binary(binary_path: str, ascii_path: str):
    with open(binary_path, "rb") as f:
        hex_str = binascii.hexlify(f.read()).decode("utf-8")

    ascii_result = bytearray.fromhex(hex_str).decode("ascii", errors="ignore")

    with open(ascii_path, "w", encoding="utf-8") as f:
        f.write(ascii_result)

    print(f"✅ ASCII 변환 파일 저장 완료: {ascii_path}")

In [ ]:
def ask_llm_from_ascii_file(ascii_path: str, query: str) -> str:
    with open(ascii_path, "r", encoding="utf-8") as f:
        ascii_text = f.read()

    prompt = f"""### 지시문:
아래 텍스트는 바이너리 파일을 hex dump → ASCII로 변환한 것입니다.
{query}

### 텍스트:
{ascii_text[:3500]}

### 답변:"""

    output = pipe(prompt, max_new_tokens=300, do_sample=False)
    return output[0]["generated_text"].split("### 답변:")[-1].strip()

In [ ]:
def save_result_to_csv(result: str, csv_path: str):
    with open(csv_path, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["LLM 분석 결과"])
        writer.writerow([result])
    print(f"📁 CSV 저장 완료: {csv_path}")

In [ ]:
generate_ascii_from_binary(BINARY_FILE_PATH, ASCII_DUMP_PATH)
result = ask_llm_from_ascii_file(ASCII_DUMP_PATH, QUERY)
save_result_to_csv(result, CSV_OUTPUT_PATH)